In [8]:
from src.datagen.openml import *

In [24]:
data_dict = get_openml_dataset(COMPASS_ID)
features, labels = data_dict['features'], data_dict['labels']

In [25]:
labels

0       -1
1       -1
2       -1
3       -1
4       -1
        ..
16639    0
16640    0
16641    0
16642    0
16643    0
Name: is_recid, Length: 16644, dtype: int64

In [28]:
for col in features.columns:
    print(col, features[col].unique())
    print()

sex ['1', '0']
Categories (2, object): ['0', '1']

age [25 37 33 44 24 56 31 53 27 26 32 60 21 20 52 34 28 40 39 38 50 45 23 41
 30 55 29 59 22 64 58 51 48 54 42 63 36 62 57 19 35 61 49 47 46 43 69 67
 70 65 68 76 73 18 66 77 75 74 72 80 71 79 83 78 96]

age_cat ['0', '2', '1']
Categories (3, object): ['0', '1', '2']

race ['0', '2', '3', '5', '4', '1']
Categories (6, object): ['0', '1', '2', '3', '4', '5']

juv_fel_count [ 0  2  1  3 13 10  4  5  8  9 20  6]

juv_misd_count [ 0  1  4  2  3  6  7 12  5  8 13]

juv_other_count [ 0  4  2  1  5  7  3  6 10 11  9]

priors_count [ 0  2  4  1 27 13  3  6  7 29  5 10  8 24  9 11 14 15 12 22 16 28 18 20
 17 43 19 30 25 23 21 26 35 33 39 36 38 37 31]

days_b_screening_arrest [-1.000e+00 -5.600e+01 -1.100e+01 -2.400e+01 -6.000e+00 -2.000e+00
  0.000e+00 -4.700e+01 -4.000e+00  4.400e+01  7.000e+00  5.070e+02
 -5.000e+01 -7.600e+01 -5.000e+00 -4.900e+01 -9.800e+01  9.200e+01
 -8.400e+01  1.230e+02 -1.060e+02  8.700e+01  2.000e+01 -3.000e+00
  5.30